
Open csv (hardcoded location) with a column of DOIs |
takes DOIs from dataframe |
Uses ADS API to search by DOI and return citation_count and pubdate  (publication date) |
Makes each into a list |
Adds lists to dataframe as new columns |
Outputs to csv (hardcoded location)

In [2]:
import requests
import json
import csv
import pandas as pd
import sys
import requests.packages.urllib3
import urllib
import time


requests.packages.urllib3.disable_warnings()

devkey = (open('dev_key.txt','r')).read()

path = "C:\Users\Megan\Documents\LibDataResearch\IouliProject\JMoSp_no_external_links.csv"
df = pd.read_csv(path, index_col=0)
list(df.columns.values) #lists the headers


['article',
 'Non_self_referencial_link',
 'volume',
 'pre_link_text',
 'link_text',
 'clean_link_text_not_article',
 'DOI_for_article']

In [35]:
doi = df['DOI_for_article']
citeCount = []
pubDate = []

for i in range(0, len(doi)):
    print "Record num: ", i
    aDOI = doi[i]
    #must encode doi string for url and suggested to include in quotes
    url = 'https://api.adsabs.harvard.edu/v1/search/query/?q=doi:'+'"'+urllib.quote_plus(aDOI)+'"'+'&fl=citation_count,pubdate'
    #print url
    
    #required by the API
    headers = {'Authorization': 'Bearer '+devkey}
    content = requests.get(url, headers=headers)
    try:
        results = content.json()
        k = results['response']['docs'][0]
        print results
        
        pDate = k['pubdate']
        citation = k['citation_count']

    except:
        pDate = "Record Not in ADS"
        citation = "['error code: '",sys.exc_info()[0],"]"
            
    citeCount.append(citation)
    pubDate.append(pDate)
    
    time.sleep(.1)
print citeCount
print pubDate

Record num:  0
{u'responseHeader': {u'status': 0, u'QTime': 3, u'params': {u'q': u'doi:"10.1016/j.jms.2003.08.010"', u'wt': u'json', u'fl': u'citation_count,pubdate'}}, u'response': {u'start': 0, u'numFound': 1, u'docs': [{u'citation_count': 2, u'pubdate': u'2004-01-00'}]}}
Record num:  1
{u'responseHeader': {u'status': 0, u'QTime': 3, u'params': {u'q': u'doi:"10.1016/j.jms.2003.08.008"', u'wt': u'json', u'fl': u'citation_count,pubdate'}}, u'response': {u'start': 0, u'numFound': 1, u'docs': [{u'citation_count': 9, u'pubdate': u'2004-02-00'}]}}
Record num:  2
{u'responseHeader': {u'status': 0, u'QTime': 3, u'params': {u'q': u'doi:"10.1016/j.jms.2003.08.009"', u'wt': u'json', u'fl': u'citation_count,pubdate'}}, u'response': {u'start': 0, u'numFound': 1, u'docs': [{u'citation_count': 6, u'pubdate': u'2004-01-00'}]}}
Record num:  3
{u'responseHeader': {u'status': 0, u'QTime': 2, u'params': {u'q': u'doi:"10.1016/j.jms.2003.09.004"', u'wt': u'json', u'fl': u'citation_count,pubdate'}}, u'resp

In [36]:
df['citationCount'] = citeCount
df['Date'] = pubDate

df.to_csv(path_or_buf="C:\Users\Megan\Documents\LibDataResearch\IouliProject\JMoSp_noLink_ready_for_processing.csv", encoding="utf-8")